In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import sys
import os

os.environ['MOMAPS_HOME'] = '/home/labs/hornsteinlab/Collaboration/MOmaps'

sys.path.insert(1, os.getenv('MOMAPS_HOME'))
print(f"MOMAPS_HOME: {os.getenv('MOMAPS_HOME')}")

from src.preprocessing.preprocessing_utils import rescale_intensity, fit_image_shape

%matplotlib inline

# Raw

In [ ]:
# You may change this block
batch = "batch1"
root_path = f"/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/raw/SpinningDisk/NOVA_d18_neurons_sorted/{batch}/"
n_images = 3

In [ ]:
# UTILS

def plot_images(root_folder, n_sites=3):
    for foldername, _, filenames in os.walk(root_folder):
        image_count = 0
        images = []
        names = []
        for filename in filenames:
            if filename.endswith('.tiff') or filename.endswith('.tif'):
                # if 'DMSO' not in filename:
                #     continue
                image_count += 1
                image_path = os.path.join(foldername, filename)
                image = cv2.imread(image_path, cv2.IMREAD_ANYDEPTH)
                image = fit_image_shape(image, (1024, 1024))
                rescale_image = rescale_intensity(image)
                print(rescale_image.shape)
                images.append(rescale_image)
                names.append(filename)
                if image_count == n_sites:
                    break
        if image_count == n_sites:
            plt.figure(figsize=(20, 5))
            plt.suptitle(f'{foldername}')
            for i in range(n_sites):
                plt.subplot(1, n_sites, i + 1)
                plt.imshow(images[i], cmap='gray')
                plt.axis('off')
            plt.show()


In [ ]:
plot_images(root_path, n_images)

In [ ]:
# You may change this block
batch = "batch2"
root_path = f"/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/raw/SpinningDisk/NOVA_d18_neurons_sorted/{batch}/"
n_images = 3

In [ ]:
plot_images(root_path, n_images)

# Processed

In [ ]:
processed_root_path = "/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/processed/spd2/SpinningDisk/FUS_lines_stress_2024_sorted/batch1/"#"/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/processed/spd2/SpinningDisk/240323_day29_neurons/batch1/"

In [ ]:
# UTILS

def plot_images(root_folder, cell_line=None, marker=None, n_tiles_per_marker=10):
    print(f"rootfolder = {root_folder}, cell_line = {cell_line}, marker={marker}, n_tiles_per_marker={n_tiles_per_marker}")
    for foldername, _, filenames in os.walk(root_folder):
        image_count = 0
        images = []
        images_filenames = []
        for filename in filenames:
            if filename.endswith('.npy'):
                image_path = os.path.join(foldername, filename)
                if cell_line is not None and cell_line not in image_path:
                    continue
                if marker is not None and marker != os.path.basename(foldername):
                    continue
                image_count += 1
                image = np.load(image_path)
                tile_indx = np.random.choice(np.arange(len(image)))
                image = image[tile_indx]
                images.append(image)
                images_filenames.append(f"{filename}_{tile_indx}")
                if image_count == n_tiles_per_marker:
                    break
        if image_count >= n_tiles_per_marker:
            for i in range(n_tiles_per_marker):
                channels = [os.path.basename(foldername), 'nucleus']
                fig, ax = plt.subplots(1, len(channels), figsize=(7,7))
                fig.suptitle(f'{foldername}')
                for j in range(len(channels)):
                    ax[j].imshow(images[i][...,j], cmap='gray', vmin=0, vmax=1)
                    if j == 1:
                        ax[j].set_title(f"{channels[j]}\n{images_filenames[i]}", y=0, pad=-25)
                    else:
                        ax[j].set_title(f"{channels[j]}\n{images_filenames[i]}")
                    ax[j].set_axis_off()
            plt.show()


In [ ]:
plot_images(processed_root_path, cell_line='DMSO', marker='FUS', n_tiles_per_marker=10)

In [ ]:
processed_root_path_opera = "/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/processed/Opera/batch1/"#KOLF/Untreated/"
plot_images(processed_root_path_opera, cell_line='DMSO', marker='ANXA11', n_tiles_per_marker=40)